# FunctionGemma Fine-tune cho PvZ Bot

Train AI quyết định: collect_sun, plant_pea_shooter, do_nothing

**Yêu cầu:** GPU Runtime + HuggingFace account

## 1. Cài đặt

In [ ]:
!pip install torch transformers datasets accelerate trl protobuf sentencepiece -q

## 2. Login HuggingFace

Cần token từ https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import login
login()

## 3. Load Model

In [ ]:
import torch
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers.utils import get_json_schema

BASE_MODEL = "google/functiongemma-270m-it"

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float32,
    device_map="auto",
    attn_implementation="eager"
)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

print(f"✓ Model loaded! Device: {model.device}")

## 4. Define Tools (Actions)

In [ ]:
def collect_sun(x: int, y: int) -> str:
    """Click to collect sun at pixel position."""
    return "Collected"

def plant_pea_shooter() -> str:
    """Plant a pea shooter."""
    return "Planted"

def do_nothing() -> str:
    """Wait and do nothing this turn."""
    return "Waiting"

TOOLS = [
    get_json_schema(collect_sun), 
    get_json_schema(plant_pea_shooter), 
    get_json_schema(do_nothing)
]

print("✓ Tools defined:")
for t in TOOLS:
    print(f"  - {t['function']['name']}")

## 5. Tạo Training Data

Tự động generate data cân bằng

In [ ]:
import random
from datasets import Dataset

# Generate balanced training data
raw_data = []

# COLLECT_SUN (34 samples)
for i in range(34):
    x = random.randint(100, 700)
    y = random.randint(80, 250)
    zombie = random.choice(["HAS_ZOMBIE", "NO_ZOMBIE"])
    can_plant = random.choice(["CAN_PLANT", "CANNOT_PLANT"])
    raw_data.append({
        "game_state": f"HAS_SUN x={x} y={y}. {zombie}. {can_plant}",
        "action": "collect_sun",
        "arguments": {"x": x, "y": y}
    })

# PLANT_PEA_SHOOTER (33 samples)
for i in range(33):
    raw_data.append({
        "game_state": "NO_SUN. HAS_ZOMBIE. CAN_PLANT",
        "action": "plant_pea_shooter",
        "arguments": {}
    })

# DO_NOTHING (33 samples)
for i in range(11):
    raw_data.append({"game_state": "NO_SUN. NO_ZOMBIE. CANNOT_PLANT", "action": "do_nothing", "arguments": {}})
for i in range(11):
    raw_data.append({"game_state": "NO_SUN. HAS_ZOMBIE. CANNOT_PLANT", "action": "do_nothing", "arguments": {}})
for i in range(11):
    raw_data.append({"game_state": "NO_SUN. NO_ZOMBIE. CAN_PLANT", "action": "do_nothing", "arguments": {}})

random.shuffle(raw_data)
print(f"✓ Generated {len(raw_data)} training samples")

## 6. Format Data cho Training

In [ ]:
SYSTEM_MSG = "You are a PvZ game bot. Choose ONE action based on game state."

def create_conversation(sample):
    action = sample["action"]
    args = sample["arguments"]
    
    if action == "collect_sun":
        tool_call = {"type": "function", "function": {"name": "collect_sun", "arguments": {"x": args["x"], "y": args["y"]}}}
    elif action == "plant_pea_shooter":
        tool_call = {"type": "function", "function": {"name": "plant_pea_shooter", "arguments": {}}}
    else:
        tool_call = {"type": "function", "function": {"name": "do_nothing", "arguments": {}}}
    
    return {
        "messages": [
            {"role": "developer", "content": SYSTEM_MSG},
            {"role": "user", "content": sample["game_state"]},
            {"role": "assistant", "tool_calls": [tool_call]},
        ],
        "tools": TOOLS
    }

dataset = Dataset.from_list(raw_data)
dataset = dataset.map(create_conversation, remove_columns=dataset.features, batched=False)
dataset = dataset.train_test_split(test_size=0.2, shuffle=True)

print(f"✓ Train: {len(dataset['train'])}, Test: {len(dataset['test'])}")

## 7. Training

In [ ]:
from trl import SFTTrainer, SFTConfig

args = SFTConfig(
    output_dir="pvz_functiongemma",
    max_length=512,
    packing=False,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    gradient_checkpointing=False,
    optim="adamw_torch",
    logging_steps=10,
    eval_strategy="epoch",
    learning_rate=5e-5,
    fp16=False,
    bf16=False,
    lr_scheduler_type="constant",
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer,
)

print("Starting training...")
trainer.train()
print("\n✓ Training complete!")

## 8. Test Model

In [ ]:
def test_bot(game_state):
    messages = [
        {"role": "developer", "content": SYSTEM_MSG},
        {"role": "user", "content": game_state},
    ]
    
    inputs = tokenizer.apply_chat_template(
        messages, 
        tools=TOOLS, 
        add_generation_prompt=True, 
        return_dict=True, 
        return_tensors="pt"
    )
    
    out = model.generate(
        **inputs.to(model.device), 
        pad_token_id=tokenizer.eos_token_id, 
        max_new_tokens=64
    )
    output = tokenizer.decode(out[0][len(inputs["input_ids"][0]):], skip_special_tokens=False)
    return output

print("="*50)
print("TEST PVZ BOT")
print("="*50)

test_cases = [
    "HAS_SUN x=300 y=150. NO_ZOMBIE. CAN_PLANT",
    "NO_SUN. HAS_ZOMBIE. CAN_PLANT",
    "NO_SUN. NO_ZOMBIE. CANNOT_PLANT",
    "HAS_SUN x=450 y=200. HAS_ZOMBIE. CAN_PLANT",
]

for t in test_cases:
    print(f"\n📥 Input: {t}")
    print(f"📤 Output: {test_bot(t)}")

## 9. Save & Download

In [ ]:
# Save model
model.save_pretrained("pvz_functiongemma_final")
tokenizer.save_pretrained("pvz_functiongemma_final")

# Zip for download
!zip -r pvz_functiongemma_final.zip pvz_functiongemma_final/

print("\n✓ Model saved! Download pvz_functiongemma_final.zip")

In [ ]:
from google.colab import files
files.download('pvz_functiongemma_final.zip')